---

<div align="center">

### **FINAL PROJECT - SIGNALS AND SYSTEMS - 2025 2S**
#### **Group members:** Martín Ramírez Espinosa, Valeria Corredor García
##### Department of Electrical, Electronic and Computer Engineering
##### National University of Colombia - Manizales Campus

</div>

---

## Objective: Implement time- and frequency-domain representation techniques for recognizing electroencephalography (EEG) signals in motor imagery tasks


![eegMI](https://figures.semanticscholar.org/288a54f091264377eccc99a19079c9387d66a78f/3-Figure2-1.png)

EEG signals can be noisy due to various sources, including physiological artifacts and electromagnetic interference. They also vary from person to person, which complicates feature extraction and understanding the signals. Moreover, this variability, influenced by genetic and cognitive factors, poses challenges for developing subject-independent solutions.

**Database**: GiGaScience Database [https://gigadb.org/dataset/100295](https://gigadb.org/dataset/100295)

See Section 3.1 in [Multimodal Explainability Using Class Activation Maps and Canonical Correlation for MI-EEG Deep Learning Classification](https://www.mdpi.com/2076-3417/14/23/11208)


## Install the necessary libraries

## Exercise 1
Investigate the purpose of the following libraries.

## Importing required libraries

## Functions needed for light preprocessing of the data

## Establish the test protocol and EEG montage configuration

Describe the data acquisition protocol and the montage used.

![mi](https://www.mdpi.com/diagnostics/diagnostics-13-01122/article_deploy/html/images/diagnostics-13-01122-g001.png)
![montaje](https://www.mdpi.com/applsci/applsci-14-11208/article_deploy/html/images/applsci-14-11208-g001.png)


## Define the path and parameters for loading the EEG data

## Load the data for the desired participant

If you want to load data for all participants, iterate through the subject list so each one is loaded individually depending on what you intend to do.

For example:

```
for i in sbj:
    X, y = load_GIGA(sbj=sbj, **load_args)
```


## Visualization of EEG signals in the time domain

# Exercise 2

Discuss the preceding plot.

Note: Discuss what brain rhythms consist of

![montaje](https://cdn.shopify.com/s/files/1/0348/7053/files/storage.googleapis.com-486681944373284_61cb9936-f6c2-493d-8402-3426d7f5a049_1024x1024.jpg?v=1689309340)


# Exercise 3

Explain how each of the five dimensions of the Xrc array was computed.

## Visualization of EEG signals in the frequency domain

## Exercise 4

Discuss the preceding plot.

## Exercise 5

Discuss the plots.

## Visualization of spectrograms

Review what the Short-Time Fourier Transform is.

# Exercise 6

Present the STFT plots for different channels across the five brain rhythms and discuss.

## Visualization of EEG signals on the 10-20 montage

# Topographic Maps

## Exercise 7

Discuss.

## Common Spatial Patterns

Investigate what Common Spatial Patterns (CSP) are and their application to EEG signal processing.